In [ ]:
import datetime as dt
import oandapy
import pandas as pd
import numpy as np

In [ ]:
# Set initial year for data y1, end year of data y2, granularity of data gran.
y1 = '2005'
y2 = '2018'
gran = 'M10'

# Open oanda client with access token in practice environmet
oanda  = oandapy.API(environment="practice",
                     access_token="")


# Set Default Values for M10
# 
# Granularity of data in pandas freq, and as integer in minutes dl_interval,
# Granularity for output format granop, interval size as tine step diff,
# offset between two different download attempts offset (5000 is maximum via oanda)
freq = '10T'
dl_interval = 10
granop = gran
diff = '0 days, 00:10:00'
offset = pd.DateOffset(minutes=4800*int(dl_interval))

## Potential customization for other intervals to write as function
# dl_interval = gran[1:]

# if gran[0] == 'M':
# 	freq = dl_interval + 'T'
# 	if len(gran) == 2:
# 		granop = 'M0' + dl_interval
# 		diff = '0 days, 00:0' + dl_interval + ':00'
# 	else:
# 		granop = 'M' + dl_interval
# 		diff = '0 days, 00:' + dl_interval + ':00'
# 	offset = pd.DateOffset(minutes=4800*int(dl_interval))
	
# elif gran[0] == 'H':
# 	freq = str(dl_interval) + 'H'
# 	if len(gran) == 2:
# 		granop = 'H0' + str(dl_interval)
# 		diff = '0 days, 0' + dl_interval + ':00:00'
# 	else:
# 		granop = 'H' + str(dl_interval)
# 		diff = '0 days, ' + dl_interval + ':00:00'
# 	offset = pd.DateOffset(hours=4800*int(dl_interval))

# elif gran[0] == 'S':
# 	freq = str(dl_interval) + 'S'
# 	if len(gran) == 2:
# 		granop = 'S0' + dl_interval
# 		diff = '0 days, 00:00:0' + dl_interval
# 	else:
# 		granop = 'S' + str(dl_interval)
# 		diff = '0 days, 00:00:' + dl_interval
# 	offset = pd.DateOffset(seconds=4800*int(dl_interval))

# elif gran[0] == 'D':
# 	freq = 'D'
# 	granop = gran
# 	diff = '1 day, 00:00:00'
# 	offset = pd.DateOffset(days=4800*int(dl_interval))


# Function to download from oanda
def oanda_download(df, x0, xf, gran, instrument):
    # Each call of the function downloads data between x0 and xf that hasn't been downloaded previously.
    # To make as few requests to oanda as possible, make the interval as large as safely possible.
    # Oanda allows 5000 data points at a time; for margin we pull data from up to 4800 time points.
    # This is to account for potential duplicate data that'd throw an exception.
	if gran[0] == 'S':
		offset = pd.DateOffset(seconds=4800*int(gran[1:]))
	elif gran[0] == 'M':
		offset = pd.DateOffset(minutes=4800*int(gran[1:]))
	elif gran[0] == 'H':
		offset = pd.DateOffset(hours=4800*int(gran[1:]))
	else:
		offset = pd.DateOffset(hours=4800*int(gran[1:]))
	
    # If the end point given by user is later than current time, shorten interval.
	if dt.datetime.now() < xf:
		xf = dt.datetime.now()
	
    # Download data between x0 and x1, which corresponds to 4800 intervals
	x1 = x0 + offset
	
    # But only download until xf
	if x1 > xf:
		x1 = xf
	
	x0_str = x0.strftime('%Y-%m-%dT%H:%M:%S')
	x1_str = x1.strftime('%Y-%m-%dT%H:%M:%S')

    # Try download. 
    # If now data available for interval, skip.
	try:
		tmp = oanda.get_history(instrument=instrument, granularity=gran,
                                start=x0_str, end=x1_str).get("candles")
	except:
		tmp = {}
	
    # If data available, append to data frame.
	if tmp != {}:
		app = pd.DataFrame(tmp, columns=tmp.pop(0)).loc[:,('time', 'closeBid', 'closeAsk')]
		app.columns = ('time',  sym + ' Bid', sym + ' Ask')
		app.loc[:, 'time'] = pd.to_datetime(app['time'])
		app = app.set_index('time')
		
		df = pd.concat([df, app])

    # Set old x1 as new x1 plus one second for next step and call self again 
    # as long as xf isn't included in download.
	if x1 < xf:
		x0 = x1 + pd.DateOffset(seconds=1)
		df = oanda_download(df, x0, xf, gran, instrument)
		
	return df


# String names for file naming
exportstring = granop+'_'+y1+'_'+y2
importstring = exportstring



In [ ]:
# List of symbols to download
symbols = ["EUR_AUD","EUR_CAD","EUR_CHF","EUR_GBP","EUR_JPY", "USD_MXN",
           "EUR_NOK","EUR_NZD","EUR_SEK","EUR_SGD","EUR_USD","EUR_ZAR", "XAU_USD"]
symbolseur = ["EUR_AUD","EUR_CAD","EUR_CHF","EUR_EUR","EUR_GBP","EUR_JPY", "USD_MXN",
              "EUR_NOK","EUR_NZD","EUR_SEK","EUR_SGD","EUR_USD","EUR_ZAR"]

for sym in symbols:
    x0 = dt.datetime(int(y1), 1, 1, 0, 0, 0)
    xf = dt.datetime(int(y2),12,31,23,59,59)
    # If last date is later than current time, use current time as end point
    if dt.datetime.now() < xf:
        xf = dt.datetime.now()

    # Create data frame 
    df = pd.DataFrame(data = None, columns = ('time',  sym + ' Bid', sym + ' Ask'))
    df = df.set_index('time')
    
    # See if a file exists with previously downloaded data.
    # If it doesn't for y2, loop down to y1; if all fails download from scratch.
    yloop = int(y2)
    while yloop >= int(y1):
        try:
            importstring = granop+'_'+y1+'_'+str(yloop)
            df = pd.read_csv(importstring + '_curr_' + sym + '.csv', engine='c', index_col="time", parse_dates=True)
            last_date = df.index[-1]
            print 'Found ' + sym + ' file. xf is ' +  xf.strftime('%Y-%m-%dT%H:%M:%S') +'.'
            break
        except:
            yloop = yloop-1
            print yloop
    try:
        print 'Last date in file is ' +  last_date.strftime('%Y-%m-%dT%H:%M:%S') + '.'
    except:
        last_date = x0
        print 'Could not find or read ' + sym + ' file.'

        
    df = oanda_download(df, last_date, xf, gran, sym)
    df.loc[:, sym] = 0.5*(df.loc[:, sym + ' Bid'] + df.loc[:, sym + ' Ask'])
    df.to_csv(exportstring + '_curr_' + sym + '.csv', index=True)
    
    if sym[0:3] != "EUR":
        continue

    

    if sym == symbols[0]:
        temp = df.copy()
        # 'merged' is the dataframe in which we write all the data, so we fill it up with AUD as a starter
        merged = temp
        # Loop through for all the other currencies
    else:
        newfile = df.copy()
        # Attach a new column to the table for the currency, but only take time values which exist for all currencies (inner join).
        merged = pd.merge(merged, newfile, how='inner', left_index=True, right_index=True)
        
        

# Rename dataframe for clarity and select only midpoint columns
rates_data = merged.loc[:, symbols]
# Add a column with the EUR/EUR exchange rate
rates_data.insert(3, 'EUR_EUR', 1.)
# Sort columns alphabetically by their index
rates_data = rates_data.reindex_axis(sorted(rates_data.columns), axis=1)
# Export dataframe to a csv file. Index false indicates that we don't want to export the row numbers as well
rates_data.to_csv(exportstring+'_rates.csv')


# Export all timestamps starting with the second row because that will be the first return we calculate
pd.Series(data=rates_data.index.values, name='time').to_csv(exportstring+'_timestamps.csv', index=False, header=True)